#PROCESO DE ETL PARA EL PRIMER TRABAJO INDIVIDUAL MLOPS HENRY

### IMPORTAMOS LAS LIBRERIAS QUE VAMOS A UTILIZAR

In [ ]:
# pandas para 
import pandas as pd
# numpy para 
import numpy as np
# os para
import os
# datetime para 
from datetime import datetime

### REQUERIMIENTO SOLICITADO PARA ESTE ETL
REQ01: Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

REQ02: Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

REQ03: De haber fechas, deberán tener el formato AAAA-mm-dd

REQ04: Los campos de texto deberán estar en minúsculas, sin excepciones

REQ05: El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

### ingestamos los .CSV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_amazon  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/amazon_prime_titles.csv")
df_disney  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/disney_plus_titles.csv")
df_hulu   = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/hulu_titles.csv")
df_netflix = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/netflix_titles.csv")


### hacemos un breve analisis de la data ingestada

#### analizamos df_amazon

In [ ]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [ ]:
df_amazon.isna().sum().sort_values(ascending = False)

date_added      9513
country         8996
director        2082
cast            1233
rating           337
show_id            0
type               0
title              0
release_year       0
duration           0
listed_in          0
description        0
dtype: int64

In [ ]:
df_amazon.duplicated().sum()

0

In [ ]:
df_amazon["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [ ]:
df_amazon.shape

(9668, 12)

In [ ]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [ ]:
df_amazon.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaN,2010,R,103 min,"Action, Drama, Suspense","Harry Brown, starring two-time Academy Award w..."


#### analizamos df_disney

In [ ]:
df_disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [ ]:
df_disney.isna().sum().sort_values(ascending = False)

director        473
country         219
cast            190
date_added        3
rating            3
show_id           0
type              0
title             0
release_year      0
duration          0
listed_in         0
description       0
dtype: int64

In [ ]:
df_disney.duplicated().sum()

0

In [ ]:
df_disney["rating"].unique()

array(['TV-G', 'PG', 'TV-PG', nan, 'PG-13', 'TV-14', 'G', 'TV-Y7', 'TV-Y',
       'TV-Y7-FV'], dtype=object)

In [ ]:
df_disney.shape

(1450, 12)

In [ ]:
df_disney.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.


In [ ]:
df_disney.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1447,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum..."
1448,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...
1449,s1450,Movie,Captain Sparky vs. The Flying Saucers,Mark Waring,Charlie Tahan,United States,"April 1, 2020",2012,TV-G,2 min,"Action-Adventure, Animals & Nature, Animation",View one of Sparky's favorite home movies.


#### analizamos df_hulu

In [ ]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [ ]:
df_hulu.isna().sum().sort_values(ascending = False)

cast            3073
director        3070
country         1453
rating           520
duration         479
date_added        28
description        4
show_id            0
type               0
title              0
release_year       0
listed_in          0
dtype: int64

In [ ]:
df_hulu.duplicated().sum()

0

In [ ]:
df_hulu["rating"].unique()

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', '2 Seasons', 'TV-Y', '93 min', '4 Seasons', 'TV-Y7',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '84 min', '103 min', '105 min', '119 min', '114 min', '82 min',
       '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min',
       '111 min', '81 min', '49 min', '45 min', '41 min', '73 min',
       '40 min', '36 min', '39 min', '34 min', '47 min', '65 min',
       '37 min', '78 min', '102 min', '129 min', '115 min', '112 min',
       'NR', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min',
       '28 min', '64 min', '7 min', '5 min', '6 min', '127 min',
       '142 min', '108 min', '57 min', '118 min', '116 min', '12 Seasons',
       '71 

In [ ]:
df_hulu.shape

(3073, 12)

In [ ]:
df_hulu.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...


In [ ]:
df_hulu.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3070,s3071,TV Show,The Fades,NaN,NaN,United Kingdom,NaN,2011,TV-14,1 Season,"Horror, International, Science Fiction",Seventeen-year-old Paul is haunted by apocalyp...
3071,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...
3072,s3073,TV Show,Tokyo Magnitude 8.0,NaN,NaN,Japan,NaN,2009,TV-14,1 Season,"Anime, Drama, International",The devastation is unleashed in the span of se...


#### analizamos df_netflix

In [ ]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
df_netflix.isna().sum().sort_values(ascending = False)

director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64

In [ ]:
df_netflix.duplicated().sum()

0

In [ ]:
df_netflix["rating"].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
df_netflix.shape

(8807, 12)

In [ ]:
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [ ]:
df_netflix.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


#### conclusiones de la data analizada

df_amazon contiene nulos en:
date_added      9513
country         8996
director        2082
cast            1233
rating           337

df_disney contiene nulos en: director        473
country         219
cast            190
date_added        3
rating            3

df_hulu contiene nulos en: cast            3073
director        3070
country         1453
rating           520
duration         479
date_added        28
description        4

df_netflix contiene nulos en: director        2634
country          831
cast             825
date_added        10
rating             4
duration           3

Eliminar las columnas:  country, director, cast
Imputar las columnas: description
Analizar a mas detalle las columnas: date_added, rating para ver los cambios a realizar.


El campo "rating" en df_hulu y df_netflix tiene datos del campo "duration", hay que hacer una transformacion en el campo rating, para mover los valores con "min" y "seasons"

### hacemos un breve analisis de la data ingestada

#### analizamos df_amazon

In [ ]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [ ]:
df_amazon.isna().sum().sort_values(ascending = False)

date_added      9513
country         8996
director        2082
cast            1233
rating           337
show_id            0
type               0
title              0
release_year       0
duration           0
listed_in          0
description        0
dtype: int64

In [ ]:
df_amazon.duplicated().sum()

0

In [ ]:
df_amazon["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [ ]:
df_amazon.shape

(9668, 12)

In [ ]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [ ]:
df_amazon.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaN,2010,R,103 min,"Action, Drama, Suspense","Harry Brown, starring two-time Academy Award w..."


#### analizamos df_disney

In [ ]:
df_disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [ ]:
df_disney.isna().sum().sort_values(ascending = False)

director        473
country         219
cast            190
date_added        3
rating            3
show_id           0
type              0
title             0
release_year      0
duration          0
listed_in         0
description       0
dtype: int64

In [ ]:
df_disney.duplicated().sum()

0

In [ ]:
df_disney["rating"].unique()

array(['TV-G', 'PG', 'TV-PG', nan, 'PG-13', 'TV-14', 'G', 'TV-Y7', 'TV-Y',
       'TV-Y7-FV'], dtype=object)

In [ ]:
df_disney.shape

(1450, 12)

In [ ]:
df_disney.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.


In [ ]:
df_disney.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1447,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum..."
1448,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...
1449,s1450,Movie,Captain Sparky vs. The Flying Saucers,Mark Waring,Charlie Tahan,United States,"April 1, 2020",2012,TV-G,2 min,"Action-Adventure, Animals & Nature, Animation",View one of Sparky's favorite home movies.


#### analizamos df_hulu

In [ ]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [ ]:
df_hulu.isna().sum().sort_values(ascending = False)

cast            3073
director        3070
country         1453
rating           520
duration         479
date_added        28
description        4
show_id            0
type               0
title              0
release_year       0
listed_in          0
dtype: int64

In [ ]:
df_hulu.duplicated().sum()

0

In [ ]:
df_hulu["rating"].unique()

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', '2 Seasons', 'TV-Y', '93 min', '4 Seasons', 'TV-Y7',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '84 min', '103 min', '105 min', '119 min', '114 min', '82 min',
       '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min',
       '111 min', '81 min', '49 min', '45 min', '41 min', '73 min',
       '40 min', '36 min', '39 min', '34 min', '47 min', '65 min',
       '37 min', '78 min', '102 min', '129 min', '115 min', '112 min',
       'NR', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min',
       '28 min', '64 min', '7 min', '5 min', '6 min', '127 min',
       '142 min', '108 min', '57 min', '118 min', '116 min', '12 Seasons',
       '71 

In [ ]:
df_hulu.shape

(3073, 12)

In [ ]:
df_hulu.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...


In [ ]:
df_hulu.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3070,s3071,TV Show,The Fades,NaN,NaN,United Kingdom,NaN,2011,TV-14,1 Season,"Horror, International, Science Fiction",Seventeen-year-old Paul is haunted by apocalyp...
3071,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...
3072,s3073,TV Show,Tokyo Magnitude 8.0,NaN,NaN,Japan,NaN,2009,TV-14,1 Season,"Anime, Drama, International",The devastation is unleashed in the span of se...


#### analizamos df_netflix

In [ ]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
df_netflix.isna().sum().sort_values(ascending = False)

director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64

In [ ]:
df_netflix.duplicated().sum()

0

In [ ]:
df_netflix["rating"].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
df_netflix.shape

(8807, 12)

In [ ]:
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [ ]:
df_netflix.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


#### conclusiones de la data analizada

df_amazon contiene nulos en:
date_added      9513
country         8996
director        2082
cast            1233
rating           337

df_disney contiene nulos en: director        473
country         219
cast            190
date_added        3
rating            3

df_hulu contiene nulos en: cast            3073
director        3070
country         1453
rating           520
duration         479
date_added        28
description        4

df_netflix contiene nulos en: director        2634
country          831
cast             825
date_added        10
rating             4
duration           3

Eliminar las columnas:  country, director, cast
Imputar las columnas: description
Analizar a mas detalle las columnas: date_added, rating para ver los cambios a realizar.


El campo "rating" en df_hulu y df_netflix tiene datos del campo "duration", hay que hacer una transformacion en el campo rating, para mover los valores con "min" y "seasons"

### hacemos un breve analisis de la data ingestada

#### analizamos df_amazon

In [ ]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [ ]:
df_amazon.isna().sum().sort_values(ascending = False)

date_added      9513
country         8996
director        2082
cast            1233
rating           337
show_id            0
type               0
title              0
release_year       0
duration           0
listed_in          0
description        0
dtype: int64

In [ ]:
df_amazon.duplicated().sum()

0

In [ ]:
df_amazon["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [ ]:
df_amazon.shape

(9668, 12)

In [ ]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [ ]:
df_amazon.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaN,2010,R,103 min,"Action, Drama, Suspense","Harry Brown, starring two-time Academy Award w..."


#### analizamos df_disney

In [ ]:
df_disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [ ]:
df_disney.isna().sum().sort_values(ascending = False)

director        473
country         219
cast            190
date_added        3
rating            3
show_id           0
type              0
title             0
release_year      0
duration          0
listed_in         0
description       0
dtype: int64

In [ ]:
df_disney.duplicated().sum()

0

In [ ]:
df_disney["rating"].unique()

array(['TV-G', 'PG', 'TV-PG', nan, 'PG-13', 'TV-14', 'G', 'TV-Y7', 'TV-Y',
       'TV-Y7-FV'], dtype=object)

In [ ]:
df_disney.shape

(1450, 12)

In [ ]:
df_disney.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.


In [ ]:
df_disney.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1447,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum..."
1448,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...
1449,s1450,Movie,Captain Sparky vs. The Flying Saucers,Mark Waring,Charlie Tahan,United States,"April 1, 2020",2012,TV-G,2 min,"Action-Adventure, Animals & Nature, Animation",View one of Sparky's favorite home movies.


#### analizamos df_hulu

In [ ]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [ ]:
df_hulu.isna().sum().sort_values(ascending = False)

cast            3073
director        3070
country         1453
rating           520
duration         479
date_added        28
description        4
show_id            0
type               0
title              0
release_year       0
listed_in          0
dtype: int64

In [ ]:
df_hulu.duplicated().sum()

0

In [ ]:
df_hulu["rating"].unique()

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', '2 Seasons', 'TV-Y', '93 min', '4 Seasons', 'TV-Y7',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '84 min', '103 min', '105 min', '119 min', '114 min', '82 min',
       '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min',
       '111 min', '81 min', '49 min', '45 min', '41 min', '73 min',
       '40 min', '36 min', '39 min', '34 min', '47 min', '65 min',
       '37 min', '78 min', '102 min', '129 min', '115 min', '112 min',
       'NR', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min',
       '28 min', '64 min', '7 min', '5 min', '6 min', '127 min',
       '142 min', '108 min', '57 min', '118 min', '116 min', '12 Seasons',
       '71 

In [ ]:
df_hulu.shape

(3073, 12)

In [ ]:
df_hulu.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...


In [ ]:
df_hulu.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3070,s3071,TV Show,The Fades,NaN,NaN,United Kingdom,NaN,2011,TV-14,1 Season,"Horror, International, Science Fiction",Seventeen-year-old Paul is haunted by apocalyp...
3071,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...
3072,s3073,TV Show,Tokyo Magnitude 8.0,NaN,NaN,Japan,NaN,2009,TV-14,1 Season,"Anime, Drama, International",The devastation is unleashed in the span of se...


#### analizamos df_netflix

In [ ]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
df_netflix.isna().sum().sort_values(ascending = False)

director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64

In [ ]:
df_netflix.duplicated().sum()

0

In [ ]:
df_netflix["rating"].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
df_netflix.shape

(8807, 12)

In [ ]:
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [ ]:
df_netflix.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


#### conclusiones de la data analizada

df_amazon contiene nulos en:
date_added      9513
country         8996
director        2082
cast            1233
rating           337

df_disney contiene nulos en: director        473
country         219
cast            190
date_added        3
rating            3

df_hulu contiene nulos en: cast            3073
director        3070
country         1453
rating           520
duration         479
date_added        28
description        4

df_netflix contiene nulos en: director        2634
country          831
cast             825
date_added        10
rating             4
duration           3

Eliminar las columnas:  country, director, cast
Imputar las columnas: description
Analizar a mas detalle las columnas: date_added, rating para ver los cambios a realizar.


El campo "rating" en df_hulu y df_netflix tiene datos del campo "duration", hay que hacer una transformacion en el campo rating, para mover los valores con "min" y "seasons"

### REQ01 - SOLUCION
REQ01: Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

creamos el campo nuevo "id"

In [ ]:
df_amazon['id']  = 'a' + df_amazon['show_id']
df_disney['id']  = 'd' + df_disney['show_id']
df_hulu['id']    = 'h' + df_hulu['show_id']
df_netflix['id'] = 'n' + df_netflix['show_id']

creamos un campo "plataforma" para saber si es: amazon, disney, hulu o netflix

In [ ]:
df_amazon["plataforma"]  = "a"
df_disney["plataforma"]  = "d"
df_hulu["plataforma"]    = "h"
df_netflix["plataforma"] = "n"

creamos un campo "link_pelicula", que contendra el link directo para que el usuario pueda ver la pelicula

In [ ]:
df_amazon["link_pelicula"]  = "https://www.primevideo.com/"
df_disney["link_pelicula"]  = "https://www.disneyplus.com/es-pe"
df_hulu["link_pelicula"]    = "https://www.hulu.com/es-us/latino-tv"
df_netflix["link_pelicula"] = "https://www.netflix.com/pe/"

Visualizamos con df_amazon para ver el nuevo campo "id", "plataforma" y sus valores

In [ ]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,plataforma,link_pelicula
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,a,https://www.primevideo.com/
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2,a,https://www.primevideo.com/
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/


In [ ]:
df_amazon.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,plataforma,link_pelicula
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou...",as9666,a,https://www.primevideo.com/
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ...",as9667,a,https://www.primevideo.com/
9667,s9668,Movie,Harry Brown,Daniel Barber,"Michael Caine, Emily Mortimer, Joseph Gilgun, ...",NaN,NaN,2010,R,103 min,"Action, Drama, Suspense","Harry Brown, starring two-time Academy Award w...",as9668,a,https://www.primevideo.com/


### REQ02 - SOLUCION
REQ02: Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

#### Se identificaron valores que no le correspondian al campo "rating", que deberian estar en al campo "duration", se procederá a juntar las 4 tablas y trabajar todas los REQUERIMIENTOS de forma conjunta

In [ ]:
df_peliculas = pd.concat([df_amazon, df_disney, df_hulu, df_netflix]).reset_index(drop=True)

In [ ]:
df_peliculas.shape

(22998, 15)

In [ ]:
df_peliculas.tail(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,plataforma,link_pelicula
22995,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,ns8805,n,https://www.netflix.com/pe/
22996,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",ns8806,n,https://www.netflix.com/pe/
22997,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...,ns8807,n,https://www.netflix.com/pe/


In [ ]:
limpieza_1 = df_peliculas[df_peliculas['rating'].str.contains('min', na=False)].index.to_list()
limpieza_2 = df_peliculas[df_peliculas['rating'].str.contains('Season', na=False)].index.to_list()

In [ ]:
df_peliculas["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', '2 Seasons', '93 min',
       '4 Seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 Seasons', '121 min', '88 min', '101 min',
       '1 Season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 Seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '

In [ ]:
datalim = limpieza_1 + limpieza_2

for i in datalim:
    df_peliculas.loc[i,"duration"] = df_peliculas.loc[i,"rating"]
    df_peliculas.loc[i,"rating"] = np.nan

In [ ]:
df_peliculas["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', 'UR'], dtype=object)

In [ ]:
df_peliculas["rating"] = df_peliculas["rating"].replace(np.nan,"G")

In [ ]:
df_peliculas["rating"].unique()

array(['G', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', 'UR'], dtype=object)

no se menciona nada en los REQUERIMIENTOS sobre los datos con valores: 'UNRATED', 'NOT_RATE', 'NOT RATED', por lo que los dejamos así.
De ser el caso que deban tener el valor 'G' se añadiria el codigo para hacer .replace a esos 3 valores

### REQ03 - SOLUCION
REQ03: De haber fechas, deberán tener el formato AAAA-mm-dd

In [ ]:
df_peliculas["date_added"] = pd.to_datetime(df_peliculas["date_added"], infer_datetime_format=True, errors = "coerce")

In [ ]:
df_peliculas.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,plataforma,link_pelicula
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,G,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,a,https://www.primevideo.com/
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2,a,https://www.primevideo.com/
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2021-03-30,2017,G,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/


###REQ04 - SOLUCION
REQ04: Los campos de texto deberán estar en minúsculas, sin excepciones

In [ ]:
df_peliculas = df_peliculas.applymap(lambda x: x.lower() if isinstance(x,str) else x)

In [ ]:
df_peliculas.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,plataforma,link_pelicula
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,a,https://www.primevideo.com/
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,a,https://www.primevideo.com/
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/


###REQ05 - SOLUCION
REQ05: El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [ ]:
df_peliculas["duration"].unique()

array(['113 min', '110 min', '74 min', '69 min', '45 min', '52 min',
       '98 min', '131 min', '87 min', '92 min', '88 min', '93 min',
       '94 min', '46 min', '96 min', '1 season', '104 min', '62 min',
       '50 min', '3 seasons', '2 seasons', '86 min', '36 min', '37 min',
       '103 min', '9 min', '18 min', '14 min', '20 min', '19 min',
       '22 min', '60 min', '6 min', '54 min', '5 min', '84 min',
       '126 min', '125 min', '109 min', '89 min', '85 min', '56 min',
       '40 min', '111 min', '33 min', '34 min', '95 min', '99 min',
       '78 min', '4 seasons', '77 min', '55 min', '53 min', '115 min',
       '58 min', '49 min', '135 min', '91 min', '64 min', '59 min',
       '48 min', '122 min', '90 min', '102 min', '65 min', '114 min',
       '136 min', '70 min', '138 min', '100 min', '480 min', '4 min',
       '30 min', '152 min', '68 min', '57 min', '7 seasons', '31 min',
       '151 min', '149 min', '9 seasons', '141 min', '121 min', '79 min',
       '140 min', '51 min'

In [ ]:
df_peliculas[["duration_int", "duration_type"]] = df_peliculas.duration.str.split(pat=' ',expand=True )

In [ ]:
df_peliculas.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,plataforma,link_pelicula,duration_int,duration_type
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,a,https://www.primevideo.com/,113,min
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,a,https://www.primevideo.com/,110,min
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/,74,min


trabajamos con el campo nuevo "duration_int"

In [ ]:
df_peliculas["duration_int"].count()

22811

In [ ]:
df_peliculas["duration_int"].isna().sum()

187

como encontramos que existen nulos les colocamos valor de 0

In [ ]:
df_peliculas["duration_int"].fillna("0", inplace=True)

In [ ]:
df_peliculas["duration_int"].isna().sum()

0

cambiamos el tipo de la variable nueva 'duration_int' para que sea integer (int)

In [ ]:
df_peliculas["duration_int"].dtype

dtype('O')

In [ ]:
df_peliculas["duration_int"] = df_peliculas["duration_int"].astype("int")

In [ ]:
df_peliculas["duration_int"].dtype

dtype('int64')

trabajamos con el campo nuevo "duration_type"

In [ ]:
df_peliculas["duration_type"].unique()

array(['min', 'season', 'seasons', nan], dtype=object)

In [ ]:
df_peliculas["duration_type"].fillna("sin dato", inplace=True)

In [ ]:
df_peliculas["duration_type"].replace("season", "seasons", inplace=True)

In [ ]:
df_peliculas["duration_type"].unique()

array(['min', 'seasons', 'sin dato'], dtype=object)

In [ ]:
df_peliculas["duration_type"].dtype

dtype('O')

In [ ]:
df_peliculas["duration_type"] = df_peliculas["duration_type"].astype("str")

In [ ]:
df_peliculas["duration_type"].dtype

dtype('O')

###ELIMINAMOS LAS COLUMNAS QUE YA NO VAMOS A *UTILIZAR*

no tenemos informacion de si eliminar las siguientes columnas:
director
country
cast 
pero dado que no los vamos a utilizar en el desarrollo de las consultas de la API, vamos a eliminarlas. En caso de usar alguno de esos campos despues, colocar el codigo correspondiente en comentarios y volver a ejecutar todo (CTRL + F9 en el COLAB)

tampoco se dice nada del campo "date_added" que tienes valores vacios

hay 4 registros vacios en el campo Description, los imputamos con "sin dato"

In [ ]:
df_peliculas.isna().sum().sort_values(ascending = False)

country          11499
date_added        9642
director          8259
cast              5321
duration           187
description          4
show_id              0
duration_int         0
link_pelicula        0
plataforma           0
id                   0
rating               0
listed_in            0
type                 0
release_year         0
title                0
duration_type        0
dtype: int64

In [ ]:
df_peliculas["description"].fillna("sin dato", inplace=True)

In [ ]:
df_peliculas.isna().sum().sort_values(ascending = False)

country          11499
date_added        9642
director          8259
cast              5321
duration           187
show_id              0
description          0
duration_int         0
link_pelicula        0
plataforma           0
id                   0
rating               0
listed_in            0
type                 0
release_year         0
title                0
duration_type        0
dtype: int64

In [ ]:
df_peliculas.drop("director", axis=1, inplace=True)
df_peliculas.drop("country", axis=1, inplace=True)
#df_peliculas.drop("cast", axis=1, inplace=True)  se omite columna porque si se usa en el desarrollo del API

df_peliculas.drop("duration", axis=1, inplace=True)
df_peliculas.drop("show_id", axis=1, inplace=True)

In [ ]:
df_peliculas.isna().sum().sort_values(ascending = False)

date_added       9642
cast             5321
type                0
title               0
release_year        0
rating              0
listed_in           0
description         0
id                  0
plataforma          0
link_pelicula       0
duration_int        0
duration_type       0
dtype: int64

In [ ]:
df_peliculas.shape

(22998, 13)

###GUARDAMOS .CSV LIMPIO

In [ ]:
#df_peliculas.to_csv("df_peliculas.csv", index=False) 
#df_peliculas.to_csv("df_peliculas.csv",encoding="UTF8")

# antes de guardar el .csv final, hay que procesar las tablas de ratings y añadirla al dataframe para recien poder guardarla.
# Se pone en comentario esta linea hasta que se guarde como final.

#df_peliculas.to_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/peliculas_final.csv", index=False)


# PROCESAMIENTO DE LAS TABLAS DE RATINGS

ingestamos todas kas tablas de ratings

In [ ]:
rating1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/1.csv', sep=',', encoding='latin-1')
rating2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/2.csv', sep=',', encoding='latin-1')
rating3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/3.csv', sep=',', encoding='latin-1')
rating4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/4.csv', sep=',', encoding='latin-1')
rating5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/5.csv', sep=',', encoding='latin-1')
rating6 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/6.csv', sep=',', encoding='latin-1')
rating7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/7.csv', sep=',', encoding='latin-1')
rating8 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/ratings/8.csv', sep=',', encoding='latin-1')



In [ ]:
print(' rating 1', rating1.shape,'\n','rating 2', rating2.shape,'\n','rating 3', rating3.shape,'\n','rating 4', rating4.shape,'\n','rating 5', rating5.shape,'\n','rating 6', rating6.shape,'\n','rating 7', rating7.shape,'\n','rating 8', rating8.shape,'\n')


 rating 1 (1500000, 4) 
 rating 2 (1500000, 4) 
 rating 3 (1500000, 4) 
 rating 4 (1500000, 4) 
 rating 5 (1500000, 4) 
 rating 6 (1500000, 4) 
 rating 7 (524289, 4) 
 rating 8 (1500000, 4) 



unimos todos los datasets de ratings para empezar a trabajarlos

In [ ]:
df_rating = pd.concat([rating1, rating2, rating3, rating4, rating5, rating6, rating7, rating8])
df_rating.shape

(11024289, 4)

In [ ]:
df_rating.isna().sum()

userId       0
rating       0
timestamp    0
movieId      0
dtype: int64

como necesitamos obtener el promedio, lo hacemos con otro dataset, para despues unirlo con el anterior dataset

In [ ]:
df_rating_mean = df_rating.groupby(['movieId']).mean()
df_rating_mean

,userId,rating,timestamp
movieId,,,
as1,99178.623506,3.467131,1.173241e+09
as10,92915.883041,3.439571,1.176385e+09
as100,83483.262791,3.609302,1.165458e+09
as1000,90606.197938,3.556701,1.168386e+09
as1001,92349.085288,3.585288,1.179517e+09
...,...,...,...
ns995,86366.176339,3.515625,1.176070e+09
ns996,94741.060729,3.626518,1.159029e+09
ns997,86417.856842,3.530526,1.162038e+09


unimos por id de la pelicula los dos datasets generados

In [ ]:
df_peliculas_mean = pd.merge(left=df_peliculas, right=df_rating_mean, how='left', left_on='id', right_on='movieId')
df_peliculas_mean.head()

,type,title,cast,date_added,release_year,rating_x,listed_in,description,id,plataforma,link_pelicula,duration_int,duration_type,userId,rating_y,timestamp
0,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,a,https://www.primevideo.com/,113,min,99178.623506,3.467131,1.173241e+09
1,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,a,https://www.primevideo.com/,110,min,93837.476673,3.548682,1.168607e+09
2,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/,74,min,84095.315166,3.500000,1.176941e+09
3,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,a,https://www.primevideo.com/,69,min,88177.389006,3.538055,1.167142e+09
4,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,a,https://www.primevideo.com/,45,min,83381.165966,3.478992,1.167345e+09


procedemos a limpiar el datasets final

In [ ]:
#df_peliculas_mean.describe()
#userId	rating_y	timestamp

df_peliculas_mean.drop(['userId'], axis=1, inplace=True)
df_peliculas_mean.drop(['timestamp'], axis=1, inplace=True)


para facilitar cambiamos el campo 'rating_y' por el nombre 'score'

In [ ]:
df_peliculas_mean.rename(columns={'rating_y':'score'}, inplace=True)
df_peliculas_mean

,type,title,cast,date_added,release_year,rating_x,listed_in,description,id,plataforma,link_pelicula,duration_int,duration_type,score
0,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,a,https://www.primevideo.com/,113,min,3.467131
1,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,a,https://www.primevideo.com/,110,min,3.548682
2,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/,74,min,3.500000
3,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,a,https://www.primevideo.com/,69,min,3.538055
4,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,a,https://www.primevideo.com/,45,min,3.478992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,movie,zodiac,"mark ruffalo, jake gyllenhaal, robert downey j...",2019-11-20,2007,r,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",ns8803,n,https://www.netflix.com/pe/,158,min,3.438998
22994,tv show,zombie dumb,NaN,2019-07-01,2018,tv-y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",ns8804,n,https://www.netflix.com/pe/,2,seasons,3.515947
22995,movie,zombieland,"jesse eisenberg, woody harrelson, emma stone, ...",2019-11-01,2009,r,"comedies, horror movies",looking to survive in a world taken over by zo...,ns8805,n,https://www.netflix.com/pe/,88,min,3.420945
22996,movie,zoom,"tim allen, courteney cox, chevy chase, kate ma...",2020-01-11,2006,pg,"children & family movies, comedies","dragged from civilian life, a former superhero...",ns8806,n,https://www.netflix.com/pe/,88,min,3.588050


redondeamos el campo de score

In [ ]:
df_peliculas_mean = df_peliculas_mean.round(1)
df_peliculas_mean

,type,title,cast,date_added,release_year,rating_x,listed_in,description,id,plataforma,link_pelicula,duration_int,duration_type,score
0,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,a,https://www.primevideo.com/,113,min,3.5
1,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,a,https://www.primevideo.com/,110,min,3.5
2,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,a,https://www.primevideo.com/,74,min,3.5
3,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,a,https://www.primevideo.com/,69,min,3.5
4,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,a,https://www.primevideo.com/,45,min,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,movie,zodiac,"mark ruffalo, jake gyllenhaal, robert downey j...",2019-11-20,2007,r,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",ns8803,n,https://www.netflix.com/pe/,158,min,3.4
22994,tv show,zombie dumb,NaN,2019-07-01,2018,tv-y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",ns8804,n,https://www.netflix.com/pe/,2,seasons,3.5
22995,movie,zombieland,"jesse eisenberg, woody harrelson, emma stone, ...",2019-11-01,2009,r,"comedies, horror movies",looking to survive in a world taken over by zo...,ns8805,n,https://www.netflix.com/pe/,88,min,3.4
22996,movie,zoom,"tim allen, courteney cox, chevy chase, kate ma...",2020-01-11,2006,pg,"children & family movies, comedies","dragged from civilian life, a former superhero...",ns8806,n,https://www.netflix.com/pe/,88,min,3.6


ahora si vamos a generar el .csv final... antes vemos info de su data

In [ ]:
df_peliculas_mean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 22997
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   type           22998 non-null  object        
 1   title          22998 non-null  object        
 2   cast           17677 non-null  object        
 3   date_added     13356 non-null  datetime64[ns]
 4   release_year   22998 non-null  int64         
 5   rating_x       22998 non-null  object        
 6   listed_in      22998 non-null  object        
 7   description    22998 non-null  object        
 8   id             22998 non-null  object        
 9   plataforma     22998 non-null  object        
 10  link_pelicula  22998 non-null  object        
 11  duration_int   22998 non-null  int64         
 12  duration_type  22998 non-null  object        
 13  score          22998 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(10)
memory usage: 2

In [ ]:
df_peliculas_mean.to_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/peliculas_final.csv", index=False, encoding="UTF8")
# se coloco encoding="UTF8 porque dio problemas en el API

PROCESAMOS EL DATASET SCORE PARA TENERLO LISTO PARA EL EDA

In [ ]:
#df_rating contiene todas las tablas ratingx...
#df_rating = pd.concat([rating1, rating2, rating3, rating4, rating5, rating6, rating7, rating8])
df_rating.shape

(11024289, 4)

In [ ]:
num_regs_inicial = df_rating.shape[0]
num_regs_inicial

11024289

In [ ]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  int64  
 3   movieId    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 420.5+ MB


In [ ]:
df_rating.isnull().sum()

userId       0
rating       0
timestamp    0
movieId      0
dtype: int64

In [ ]:
df_rating.head()

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


notamos que debemos cambiar el formato del campo fecha: 'timestamp'
NOTA: va a demorar aprox:
porque el dataset tiene: (11024289) registros
demoro: 1 minuto y 6 segundos

In [ ]:
df_rating['timestamp'] = pd.to_datetime(df_rating['timestamp'], unit='s').dt.strftime('%Y-%m-%d')
df_rating

,userId,rating,timestamp,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500
...,...,...,...,...
1499995,124380,4.5,2007-12-04,ns5272
1499996,124380,2.5,2007-12-04,ns5492
1499997,124380,3.5,2007-12-04,hs305
1499998,124380,4.5,2007-12-04,ns7881


sabemos por el paso anterior que no hay nulos, pero y duplicados?

In [ ]:
df_rating.duplicated().sum()

10466

In [ ]:
cant_duplicados = df_rating.duplicated().sum()
cant_duplicados

10466

eliminamos los resgitros duplicados del dataset

In [ ]:
df_rating.drop_duplicates(inplace=True)

In [ ]:
num_regs_final = df_rating.shape[0]
num_regs_final

11013823

In [ ]:
print('cantidad de regitros antes de eliminar duplicados: ', num_regs_inicial)

print(f"registros eliminamos {cant_duplicados} + los registros finales {num_regs_final} , nos da un total registros de : ",  num_regs_final+ cant_duplicados)

cantidad de regitros antes de eliminar duplicados:  11024289
registros eliminamos 10466 + los registros finales 11013823 , nos da un total registros de :  11024289


verificamos estructura del dataset 'df_rating' antes de generar el .csv

In [ ]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11013823 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  object 
 3   movieId    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 420.1+ MB


puede tomar un tiempo
demoro: 39 segundos

In [ ]:
df_rating.to_csv("/content/drive/MyDrive/Colab Notebooks/PROYECTO INDIVIDUAL FASE 1/MLOpsReviews/rating_final.csv", index=False, encoding="UTF8")

# FIN DE PROCESO DE ETL - PI 01 